#### Load modules

In [2]:
import pandas as pd
import sys, os
from gpt4all import GPT4All
from tqdm import tqdm
import re
from deep_translator import GoogleTranslator

sys.path.append("\\".join(os.getcwd().split("\\")[:-1]))
%load_ext autoreload
%autoreload 2
from utils import *
from scraper import utils

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\torch\__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)


#### Import data and language model

In [2]:
data = pd.read_pickle(os.path.join("/".join(os.getcwd().split("\\")[:-1]),
                                   'preprocessing','data/merged_data.pkl'))
# define the language model (Llama with 8 billion parameters is about 4.66 GB)
model = GPT4All('Meta-Llama-3-8B-Instruct.Q4_0.gguf')

#### Clean the input database

In [4]:
language_dict = {'english':('EN', 'ENG'), 'french':('FR','FRA'), 'german':('DE','DEU'), 'italian':('IT','ITA'), 'not_found':('NA','NAN')}
data['lang_3'] = data.apply(lambda row: language_dict[detect_language(row['abstract'], not_found=True)][1], axis=1)
data['lang_2'] = data.apply(lambda row: language_dict[detect_language(row['abstract'], not_found=True)][0], axis=1)
# for idx, row in data.iterrows():
#     if row['abstract'] != 'nan':
#         if language_dict[detect_language(row['abstract_it'])][1] != 'ITA':
#             trnd = GoogleTranslator(source='auto', target='ita').translate(row['abstract'])
#             data.loc[idx, 'abstract_it'] = trnd
#         if language_dict[detect_language(row['abstract_de'])][1] != 'DEU':
#             trnd = GoogleTranslator(source='auto', target='de').translate(row['abstract'])
#             data.loc[idx, 'abstract_de'] = trnd
#         if language_dict[detect_language(row['abstract_fr'])][1] != 'FRA':
#             trnd = GoogleTranslator(source='auto', target='fr').translate(row['abstract'])
#             data.loc[idx, 'abstract_fr'] = trnd
#         if language_dict[detect_language(row['abstract_en'])][1] != 'ENG':
#             trnd = GoogleTranslator(source='auto', target='en').translate(row['abstract'])
#             data.loc[idx, 'abstract_en'] = trnd

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for 

#### Extract keywords with a local Llama model

In [13]:
data['abstract_w_count'] = data.apply(lambda x: len(x['abstract'].split(' ')), axis=1)
data.apply(lambda x: 'nan' if x['abstract'].startswith(('??','Es werden die Daten im Zeitraum','Link zu Metadaten:',
                                                'https:',
                                                'geo@bs.ch',
                                                'info.geoportal@be.ch',
                                                'sit@jura.ch',
                                                'geodaten@sg.ch',
                                                'info@example.com',
                                                'agi@tg.ch',
                                                'mail@lisag.ch',
                                                'gis@bd.zh.ch',
                                                'info.diffusion@vd.ch',
                                                'webgis@swisstopo.ch'))
                                                else x['abstract'], axis=1)
len(data[(data['abstract'] != 'nan') &
     (data['abstract'] != data['title']) &
     (data['abstract_w_count']>4)])

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for 

5939

In [15]:
# iterate through the dataframe
task = "Extract a list of key words comma separated without adjectives in original language from the following text: "
en_trns_task = "Translate the following list of words into a comma separated list in English: "
de_trns_task = "Übersetze die folgende Liste von Wörtern in eine durch Komma getrennte Liste auf Deutsch: "
fr_trns_task = "Traduisez la liste de mots suivante en une liste séparée par des virgules en francais: "
it_trns_task = "Traduci il seguente elenco di parole in un elenco separato da virgole in italiano: "

df_kg = pd.DataFrame()

for idx, row in tqdm(data.iterrows()):
    if idx > 100:
        continue
    trns_dict = {"ENG":en_trns_task, "DEU":de_trns_task, "ITA":it_trns_task,'FRA':fr_trns_task}
    if row['lang_3'] != 'NAN':
        del trns_dict[row['lang_3']]
    # filter rows with no abstract and use title instead
    if row['abstract'] == 'nan' or row['abstract']==row['title'] or len(row['abstract'].split(' '))<5 or row['abstract'].startswith(('??',
                                                                                                                                      'Es werden die Daten im Zeitraum',
                                                                                                                                      'Link zu Metadaten:',
                                                                                                                                      'https:',
                                                                                                                                      'geo@bs.ch',
                                                                                                                                      'info.geoportal@be.ch',
                                                                                                                                      'sit@jura.ch',
                                                                                                                                      'geodaten@sg.ch',
                                                                                                                                      'info@example.com',
                                                                                                                                      'agi@tg.ch',
                                                                                                                                      'mail@lisag.ch',
                                                                                                                                      'gis@bd.zh.ch',
                                                                                                                                      'info.diffusion@vd.ch',
                                                                                                                                      'webgis@swisstopo.ch')):
        # llama_answer = ask_llama(task, row['title'], idx)
        pass
    else:
        # generate LLM response with abstract and interpret it
        kwds = read_keyowrds(ask_llama(task, row['abstract']))

        df_kg = collect_keywords(kwds, row['lang_3'], trns_dict, df_kg)
        # for lang in [k for k in trns_dict.keys()]:
        #     llama_translation = ask_llama(trns_dict[lang], ", ".join(kwds), idx, check_response=False)
        #     translations[lang] = read_translation(llama_translation)
        # if check_length(translations, len(kwds)):
        #     df = pd.DataFrame({row['lang_3']:kwds})
        #     for lang in [k for k in trns_dict.keys()]:
        #         df[lang] = translations[lang]
        #         # data.loc[idx, 'kg_'+lang] = ','.join(translations[lang])
        #     # data.loc[idx, 'kg_'+row['lang_3']] = ','.join(kwds)
        #     df_kg = pd.concat([df_kg, df], axis=0, ignore_index=True)
        # else:
        #     print("Differing translation lengths!")

remove_rows = df_kg.applymap(lambda x: len(x.split(" "))).apply(sum, axis=1).loc[lambda x:x==4].index.tolist()
df_kg.drop(remove_rows, inplace=True)

0it [00:00, ?it/s]c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
2it [03:39, 111.17s/it]

Skipping differing translation lengths!


61it [1:12:09, 45.98s/it]

Skipping differing translation lengths!


75it [1:25:45, 56.59s/it]

...

87it [1:42:45, 84.20s/it] 

Skipping differing translation lengths!


88it [1:44:24, 88.48s/it]

Skipping differing translation lengths!


39395it [1:57:41,  5.58it/s] 


#### Clean the resulting dataframe

In [14]:
[k for k in range(0, 10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [23]:
df_kg = pd.read_pickle('kg_data.pkl')
df_kg = filter_translations(df_kg)
for i, row in df_kg.iterrows():
    if len(row['ITA'].split()) < 2 and len(row['DEU'].split())<2 and len(row['ENG'].split())<2 and len(row['FRA'].split())<2:
        df_kg.drop(i, inplace=True)
df_kg.drop(columns=['NAN'], inplace=True)
df_kg['ITA'] = df_kg['ITA'].str.replace('.', '')
df_kg['DEU'] = df_kg['DEU'].str.replace('.', '')
df_kg['ENG'] = df_kg['ENG'].str.replace('.', '')
df_kg['FRA'] = df_kg['FRA'].str.replace('.', '')
to_drop = [2,12,16,21,20,23,24,30,37,44]
df_kg.drop(to_drop, inplace=True)
df_kg = df_kg.reset_index(drop=True)


to_drop = [5,6,19,5,5,90,91,178,173,110, 180, 202, 285, 382, 456,196, 299, 842]
to_drop.extend([k for k in range(8, 12)])
# to_drop.extend([k for k in range(8, 12)])
df_kg.drop(to_drop, inplace=True)

df_kg[180:200]

C:\Users\ma1021525\AppData\Local\Temp\ipykernel_18544\4155709230.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_kg['ITA'] = df_kg['ITA'].str.replace('.', '')
C:\Users\ma1021525\AppData\Local\Temp\ipykernel_18544\4155709230.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_kg['DEU'] = df_kg['DEU'].str.replace('.', '')
C:\Users\ma1021525\AppData\Local\Temp\ipykernel_18544\4155709230.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_kg['ENG'] = df_kg['ENG'].str.replace('.', '')
C:\Users\ma1021525\AppData\L

,DEU,ENG,ITA,FRA
193,Feuerwehrlokale,fire stations,Stazioni dei vigili del fuoco,locaux de pompiers
194,Nebenauslösestationen,auxiliary alarm stations,Stazioni di scarico ausiliarie,stations d'alarme secondaires
195,Wasseraufbereitungsanlagen,water treatment facilities,Impianti di trattamento dell'acqua,installations d'épuration de l'eau
197,swisstopo,Swiss Topographic Survey,SwissTopo,swisstopo
198,Verkehrswege,Traffic Routes,Vie di traffico,Voies de circulation
199,Datensatz,Data Set,Scheda dati,Ensemble de données
200,Jagdreviere,Hunting district,Riferimento alla caccia,Référence de chasse
201,Kantonsgebiet,Canton area,Distretto cantonale,Région cantonale
203,Tierarten,Species types,Specie animali,Espèces animales
204,Steinkataster,Stone register,Catasto delle pietre,Carte géologique


In [24]:
df_kg[df_kg['DEU'] == 'Datenquelle']

,DEU,ENG,ITA,FRA
17,Datenquelle,Data source,Fonte dati,Source de données
68,Datenquelle,Data Source,Fonte dati,source de données
125,Datenquelle,Data Source,Fonte dati,source de données
207,Datenquelle,data source,fonte dati,source de données
324,Datenquelle,Data Source,Fonte dati,Source de données
343,Datenquelle,Data Source,Fonte dati,Source de données
515,Datenquelle,Data Source,Fonte dati,source de données
544,Datenquelle,Data Source,Fonte dati,Source de données
617,Datenquelle,Data Source,Fonte dati,source de données
659,Datenquelle,Data Source,Fonte dati,source de données


In [ ]:
df_kg.to_pickle('kg_data.pkl')

#### Generate the knowledge graph

In [4]:
df_kg = pd.read_pickle('kg_data.pkl')
# for i, row in df_kg.iterrows():
#     if len(row['ITA']) < 5 or len(row['DEU'])<5 or len(row['ENG'])<5 or len(row['FRA'])<5:
#         df_kg.drop(i, inplace=True)
for i, row in df_kg.iterrows():
    print(row)

DEU      Kartierung
ENG     Cartography
ITA      Kartierung
FRA    Cartographie
NAN             NaN
Name: 0, dtype: object
DEU              Heimweiden
ENG              Homecoming
ITA    Sentieri del ritorno
FRA    Sentiments d'enfance
NAN                     NaN
Name: 2, dtype: object
DEU           Innerrhoder
ENG          Inner Holder
ITA       Rettori interni
FRA    Intérieurs du Rhin
NAN                   NaN
Name: 3, dtype: object
DEU              Alpkataster
ENG         Alpine Catalogue
ITA           Catasto alpino
FRA    Catalogue des alpages
NAN                      NaN
Name: 4, dtype: object
DEU     Perimeter
ENG     Perimeter
ITA     Perimetro
FRA    Périphérie
NAN           NaN
Name: 5, dtype: object
DEU       Amphibienvorkommen
ENG    amphibian occurrences
ITA       Presenze di anfibi
FRA    présence d'amphibiens
NAN                      NaN
Name: 7, dtype: object
DEU    Kantone
ENG    cantons
ITA    Cantoni
FRA    cantons
NAN        NaN
Name: 8, dtype: object
DEU           

In [3]:
kg = generate_knowledge_graph('kg_data.pkl', 'knowledge_graph')

KG: Filtering translations...
KG: Loading data in the knowledge graph...


In [17]:
find_nodes_by_language(kg, 'german')

['Geographischer Bezug',
 'Rasterformat',
 'Lagefixpunkte',
 'Anbaumöglichkeiten',
 'Klimatischen Voraussetzungen',
 'Geodaten',
 'Unfallstandort',
 'Schiessanlage',
 'Betriebsstandort',
 'Altablagerung',
 'Altlast',
 'Militärische Einrichtungen',
 'Tierarten',
 'Kantonsgebiet',
 'Jagdreviere',
 'Verkehrswege',
 'Berggebiet',
 'Bruchkanten',
 'Höhenkurven',
 'Höhenfixpunkte',
 'Gis',
 'Rebflächen',
 'Dzv',
 'Datensatz',
 'Direktzahlungen',
 'Hangbeiträge',
 'Lebensbedingungen',
 'Grundstücke',
 'Orthofoto',
 'Schulgemeindgrenze',
 'Katholische Kirchgemeinden',
 'Feuerschau',
 'Erhebungszeitpunkt',
 'Gemeindegewässer',
 'Gewässerbaukataster',
 'Gschv',
 '-Areale',
 'Grundwasserschutz',
 'Schutzbereiche',
 'Wasserbaugesetz',
 'Amt',
 'Gewässerqualität',
 'Oberflächengewässern',
 'Messstellen',
 'Gesamtphosphor',
 'Doc',
 'Messstelle',
 'Gng',
 'Massstab',
 'Üp',
 'Grundbuchplänen',
 'St.Gallen',
 'Gewässernetz',
 'Rechtszustand',
 'Beitragsgesuche',
 'Revitalisierungsprojekte',
 'Hochwas

In [13]:
for ref in find_nodes_by_language(kg, 'german'):
    print([k['id'] for k in kg.v(vertex=ref).inc("means").all()['result']])

['référence géographique', 'relazione geografica', 'geographical reference']
['grid format', 'formato a raggiera', 'format raster', 'formato a rete', 'raster format']
['points fixes de niveau', 'punti di riferimento per la gestione delle lagune', 'lagefixpunktes']
['possibilités de culture', 'possibilità di coltivazione.', 'farming possibilities.']
['conditions climatiques préalables', 'condizioni climatiche presupposte', 'climatic preconditions']
['geodonnées', 'geodati', 'geodetic data']
["lieu d'accident", "luogo dell'incidente", 'accident location']
['installation à feu', 'installazione di tiro', 'shooting range']
["site d'exploitation", 'posizione operativa', 'operating location']
['déchalandage ancien', 'deposito di rifiuti', 'landfill']
['ancienne pollution', 'rischio ambientale', 'hazardous waste site']
['installations militaires', 'installazioni militari', 'military installations']
['espèces animales.', 'specie animali', 'species types']
['région cantonale', 'distretto cantona

In [4]:
find_translation(kg, 'Gefahrenkarte', 'german')

["cartes d'alerte",
 'grafico di pericoli',
 'carta di pericoli',
 'danger maps',
 'danger chart.',
 'mappa dei pericoli.',
 'mappa dei pericoli',
 'danger map.',
 'carte de danger',
 "carte d'alerte aux dangers",
 'hazard map',
 "carte d'aléa",
 'carta di pericolo',
 'danger map',
 'géocarte dangereuse',
 'carta dei pericoli',
 "carte d'alerte",
 'Gefahrenkarte']

In [10]:
kg.v("carta dei pericoli").tag("from").out("means").tag("to").view("synonym").render()